In [62]:
import os
import csv
import numpy as np
from sklearn.naive_bayes import MultinomialNB
import spacy
from sklearn.metrics import classification_report, f1_score
import time
from __future__  import unicode_literals
from io import open

In [63]:
begin_time= time.time()

In [118]:
nlp = spacy.load('en')
hockey_train = "C:/Users/harsh/Desktop/ucf academics/Sem-3/6307- Text Mining/HW2/20news-bydate.tar/20news-bydate-train/rec.sport.hockey"
auto_train = "C:/Users/harsh/Desktop/ucf academics/Sem-3/6307- Text Mining/HW2/20news-bydate.tar/20news-bydate-train/rec.autos"

In [119]:
hockey_test= "C:/Users/harsh/Desktop/ucf academics/Sem-3/6307- Text Mining/HW2/20news-bydate.tar/20news-bydate-test/rec.sport.hockey"

In [120]:
auto_test= "C:/Users/harsh/Desktop/ucf academics/Sem-3/6307- Text Mining/HW2/20news-bydate.tar/20news-bydate-test/rec.autos"

In [121]:
counts_feat = {}
words_uniq = {}
dic_feat = {}
k, j = 0, 0

In [122]:
#vocabolary list using training set and tokenise words using space

In [124]:
pos = 0
for nr_pos, name_file in enumerate(os.listdir(hockey_train)):
    doc_file = nlp(open(hockey_train + "/" + name_file, encoding="ISO-8859-1").read())
    pos += 1
    for token in doc_file:
        if token.text.isalpha() and not token.is_stop:
            if token.tag_ not in words_uniq:
                words_uniq[token.pos_] = j
                j += 1
            if token.lemma_ not in counts_feat:
                counts_feat[token.lemma_] = 0
                dic_feat[token.lemma_] = k
                k += 1
            else:
                counts_feat[token.lemma_] += 1

In [126]:
neg = 0
for nr_neg, file_name in enumerate(os.listdir(auto_train)):
    doc_file = nlp(open(auto_train + "/" + file_name, encoding="ISO-8859-1").read())
    neg += 1
    for token in doc_file:
        if token.text.isalpha() and not token.is_stop:
            if token.tag_ not in words_uniq:
                words_uniq[token.pos_] = j
                j += 1
            if token.lemma_ not in counts_feat:
                counts_feat[token.lemma_] = 0
                dic_feat[token.lemma_] = k
                k += 1
            else:
                counts_feat[token.lemma_] += 1

In [127]:
dic_feat = {}
k = 0
for key in counts_feat.keys():
    dic_feat[key] = k
    k += 1

features_total = len(counts_feat)
train_instances = pos + neg

In [129]:
train_x = np.zeros((train_instances, features_total), dtype=float)
train_y = np.vstack((np.zeros([pos, 1], dtype=int), np.ones([neg, 1], dtype=int)))

pos = 0
for nr_pos, file_name in enumerate(os.listdir(hockey_train)):
    doc_file = nlp(open(hockey_train + '/' + file_name, encoding="ISO-8859-1").read())
    for token in doc_file:
        if token.text.isalpha() and not token.is_stop:
            if token.lemma_ in dic_feat:
                train_x[pos, dic_feat[token.lemma_]] += 1
    pos += 1

neg = 0
for nr_neg, file_name in enumerate(os.listdir(auto_train)):
    doc_file = nlp(open(auto_train + '/' + file_name, encoding="ISO-8859-1").read())
    for token in doc_file:
        if token.text.isalpha() and not token.is_stop:
            if token.lemma_ in dic_feat:
                train_x[pos + neg, dic_feat[token.lemma_]] += 1
    neg += 1

In [130]:
#test dataset

In [133]:
pos = len(os.listdir(hockey_test)) #no. of test data for hockey (positive)
neg = len(os.listdir(auto_test)) #no. of test data for autos (negative)

test = pos + neg

test_x = np.zeros([test, total_features], dtype=float)
test_y = np.vstack((np.zeros([pos, 1], dtype=int), np.ones([neg, 1], dtype=int)))

In [134]:
# test matrix creation using vocab list

In [137]:
pos = 0
for nr_pos, file_name in enumerate(os.listdir(hockey_test)):
    doc_file = nlp(open(hockey_test + "/" + file_name, encoding="ISO-8859-1").read())
    for token in doc_file:
        if token.text.isalpha() and not token.is_stop:
            if token.lemma_ in dic_feat:
                test_x[pos, dic_feat[token.lemma_]] += 1
    pos += 1

neg = 0
for nr_neg, file_name in enumerate(os.listdir(auto_test)):
    doc_file = nlp(open(auto_test + "/" + file_name, encoding="ISO-8859-1").read())
    for token in doc_file:
        if token.text.isalpha() and not token.is_stop:
            if token.lemma_ in dic_feat:
                test_x[pos + neg, feat_dict[token.lemma_]] += 1
    neg += 1

In [138]:
order_new = np.arange(test)
np.random.seed(0)
np.random.shuffle(order_new)
test_x, test_y = test_x[order_new, :], test_y[order_new, :]

In [139]:
#naive bayes classifer

In [142]:
clf = MultinomialNB()
clf.fit(train_x, train_y)
pred_y = clf.predict(test_x)
row, column = test_y.shape
target_names = ['Hockey(class 0)', 'Auto(class 1)']
print(classification_report(test_y, pred_y, target_names=target_names, digits=5))
print("F1 score is : " + str(f1_score(test_y, pred_y, average='micro')))
print()
print('Total number of test documents :' + str(test))
print('Total number of train documents :' + str(train_instances))
print("Total Time for Execution:" + str(time.time() - begin_time))
print("Total number of unique words:" + str(len(words_uniq)))

                 precision    recall  f1-score   support

Hockey(class 0)    1.00000   0.98246   0.99115       399
  Auto(class 1)    0.98263   1.00000   0.99124       396

      micro avg    0.99119   0.99119   0.99119       795
      macro avg    0.99132   0.99123   0.99119       795
   weighted avg    0.99135   0.99119   0.99119       795

F1 score is : 0.9911949685534591
()
Total number of test documents :795
Total number of train documents :1194
Total Time for Execution:5932.92400002
Total number of unique words:15
